In [1]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy 
# import libraries for plotting isosurfaces
import plotly
import plotly.graph_objs as go

In [2]:
d = np.load('/Users/haivanle/Documents/AMATH582/subdata.npy')

In [3]:
# utility for clearing output of cell as loop runs in notebook
from IPython.display import clear_output

# plot the data in time 

L = 10; # length of spatial domain (cube of side L = 2*10)
N_grid = 64; # number of grid points/Fourier modes in each direction
xx = np.linspace(-L, L, N_grid+1) #spatial grid in x dir
x = xx[0:N_grid]
y = x # same grid in y,z direction
z = x

K_grid = (2*np.pi/(2*L))*np.linspace(-N_grid/2, N_grid/2 -1, N_grid) # frequency grid for one coordinate

xv, yv, zv = np.meshgrid( x, y, z) # generate 3D meshgrid for plotting

# plot iso surfaces for every third measurement

# for j in range(0,49,3):

#   signal = np.reshape(d[:, j], (N_grid, N_grid, N_grid))
#   normal_sig_abs = np.abs(signal)/np.abs(signal).max()

#   # generate data for isosurface of the 3D data 
#   fig_data = go.Isosurface( x = xv.flatten(), y = yv.flatten(), z = zv.flatten(),
#                            value = normal_sig_abs.flatten(), isomin=0.7, isomax=0.7)

#   # generate plots
#   clear_output(wait=True) # need this to discard previous figs
#   fig = go.Figure( data = fig_data )
#   fig.show()


In [8]:
# Defines a wavenumber array using the Fourier series 
k = (2 * np.pi / (2 * L)) * np.concatenate((np.arange(0, 32), np.arange(-32, 0)), axis=0)
# Performs a Fourier shift on the wavenumber array
ks = np.fft.fftshift(k)

# Create meshgrids in x, y, and z directions
Kx, Ky, Kz = np.meshgrid(ks, ks, ks)
# Creates meshgrids for the spatial coordinates X, Z, and Y
[X, Z, Y] = np.meshgrid(k, k, k)

# Computes the Fourier transform of the data for each time step and sums them up
Uk = np.sum(np.fft.fftn(d[:, j].reshape(N_grid, N_grid, N_grid, order='F')) for j in range(49))

# Finds the indexes of the maximum value in the averaged Fourier transform
indexes = np.unravel_index(np.argmax(Uk / 49), (Uk / 49).shape)
x_, y_, z_ = X[indexes], Y[indexes], Z[indexes]

/var/folders/6_/w9g0lyhn5632jr666js3y_lw0000gn/T/ipykernel_79310/3707580413.py:10: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  Uk = np.sum(np.fft.fftn(d[:, j].reshape(N_grid, N_grid, N_grid, order='F')) for j in range(49))


In [9]:
# print('Center frequency: ', x_, y_, z_)    

Center frequency:  5.340707511102648 2.199114857512855 -6.911503837897545


In [14]:
# Simple Gaussian Filter centered at the maximum frequency point
gaussian_filter = np.exp(-0.1*((X - X[indexes])**2 + (Y - Y[indexes])**2 +(Z - Z[indexes])**2))

In [15]:
# Initializes arrays to store x, y, and z positions for each time step
x_pos = np.zeros(49)
y_pos = np.zeros(49)
z_pos = np.zeros(49)

# Apply Gaussian Filter

for t in range(49):
    Un = np.reshape(d[:, t], (N_grid, N_grid, N_grid), order = 'F') # Reshapes the 1D data at time step t into a 3D array
    fft_Un = scipy.fft.ifftn((scipy.fft.fftn(Un))*gaussian_filter) # Applies the 3D Fourier transform to the reshaped data and applies the Gaussian filter
    abs_fft_Un = np.abs(fft_Un) # Takes the absolute value of the Fourier-transformed data
    max_val = np.argmax(abs_fft_Un) # Finds the index of the maximum value
    indexes1 = np.unravel_index(max_val, Un.shape) # Obtain the position in the reshaped 3D array
    # Stores the x, y, and z positions 
    x_pos[t] = x[indexes1[1]]
    y_pos[t] = y[indexes1[2]]
    z_pos[t] = z[indexes1[0]]

In [20]:
# Plot the 3d submarine path
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(x_pos, y_pos, z_pos, 'b', label='With Gaussian filter')

# Mark the initial position (first point)
ax.scatter(x_pos[0], y_pos[0], z_pos[0], c='g', marker='o', s=50, label='Initial Position')

# Mark the final position (last point)
ax.scatter(x_pos[-1], y_pos[-1], z_pos[-1], c='r', marker='o', s=50, label='Final Position')

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.legend(loc='upper right')

# Set the view angle
# ax.view_init(elev=30, azim=-20)
plt.savefig('582hw1f1.pdf')
plt.show()

In [21]:
# Plot the x,y-coordinates of the submarine path
fig = plt.figure()
ax = plt.axes()

ax.plot(x_pos, y_pos, 'b', label='With Gaussian filter')
# Mark the initial position (first point)
ax.scatter(x_pos[0], y_pos[0], c='g', marker='o', s=50, label='Initial Position')

# Mark the final position (last point)
ax.scatter(x_pos[-1], y_pos[-1], c='r', marker='o', s=50, label='Final Position')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend(loc='upper right')
plt.savefig('582hw1f2.pdf')
plt.show()
